In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import cv2
import os
import re
import pickle
import math
from pathlib import *
import multiprocessing
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from collections import *
from itertools import *
from functools import *
from sklearn.metrics import *
from scipy.stats import *
import pandas as pd
import seaborn as sns
import hashlib
from PIL import Image
from datastatics import *
from oct_Utils import *

In [2]:
map_matrix='''[ [0  0  0  0  0  0  0  0  0  0]
                [0  0  0  46 44 43 42 0  0  0]
                [0  0  46 45 44 43 41 43 0  0]
                [0  45 46 45 45 45 42 42 41 0]
                [46 46 47 46 50 50 50 3  41 0]
                [9  9  9  6  8  6  7  3  16 0]
                [0  10 8  7  6  8  9  13 17 0]
                [0  0  8  8  10 10 12 15 0  0]
                [0  0  0  9  10 11 14 0  0  0]
                [0  0  0  0  0  0  0  0  0  0]]'''#为了凑54个点，给两个视野盲区的点都随便指向了一个2


def str_to_np_array(row, dtype=np.float):
    row = row.replace('[', '').replace(']', '')
    row = row.split(' ')
    row = filter(lambda x: x != '', row)
    row = map(lambda x: dtype(x), row)
    row = np.array(list(row))
    return row

def str_to_np_mat(s, dtype=np.float):
    s = s[1:-1]
    rows = s.split('\n')
    for index, row in enumerate(rows):
        rows[index] = str_to_np_array(row, dtype)
    rows = np.stack(rows, axis=0)
    return rows

import itertools
def calculate_position(arr):
	#拼接数组函数
    array_num = list(itertools.chain.from_iterable(arr))
    array_num = np.array(array_num)
    key = np.unique(array_num)
    result = {}
    for k in key:
        index=np.argwhere(arr == k)#返回所有符合条件的索引值，更多的方法和类参数需要看
        v=[tuple(index[i]) for i in range(0,len(index))]
        result[k] = v
    result = {key:val for key, val in result.items() if  len(val)<9}

    valid_point=[]
    for key,value in result.items():
        valid_point+=value

    return result,valid_point

num_mat=str_to_np_mat(map_matrix)
countNums,valid_point=calculate_position(num_mat)#之前statics的时候，统计的数据是用的repeat的重复数据，这次要重新处理，尽量保证模块化，工程化

In [3]:
res=reverse_dict(countNums)

In [15]:
data=pd.read_csv('data.csv')#
data['np_mat_num']=data['num'].apply(str_to_np_mat)
data['np_mat_num']=data['np_mat_num'].mask(data['eye']=='OD',data['np_mat_num'].apply(lambda x:np.flip(x,axis=1)))#all_num已经翻转过，所以数据是正常的,这里相当于
#把label直接翻转好了，后续根据input直接做就可以了
all_num=data['np_mat_num']

def split_np_matrix(all_num,data:pd.DataFrame,valid_point):
    res=np.zeros((1,100))
    for num in all_num:
        res=np.concatenate((res,np.expand_dims(num.flatten(),0)),axis=0)
    res=res[1:]
    for i in range(0,res.shape[1]):
        if (i//10,i%10) in valid_point:
            data[f'({i//10},{(i)%10})']=res[:,i]
    return data

    
new_data=split_np_matrix(all_num,data,valid_point)
#new_data.to_csv('new_data.csv')#这里用excel preview看数据就出现了19.0，但实际上通过调用columns属性就可以得出其实数据属性列是正常的


In [1]:
df=pd.read()

In [4]:
id_base = 'pid'
date = '1004'
label_type = 'num'

for model, size, pos in product(['r50'], ['380'], ['disc', 'macula']):
    exp_name = f'{date}/{pos}_{label_type}_{model}_{size}'
    df_tmp = df.copy()
    df_tmp = df_tmp[df_tmp['pos'] == pos]
    df_tmp['dataset'] = df_tmp[id_base].map(to_dataset_mapping(df_tmp[id_base].tolist(), 5))
    ensure_path(f'output/{exp_name}/exp-{id_base}/tasks')
    df_tmp.to_csv(f'output/{exp_name}/exp-{id_base}/tasks/data.csv', index=False)

NameError: name 'df' is not defined